<a href="https://colab.research.google.com/github/Steve-YJ/Colab_Exercise/blob/master/%5BAgain%5D%5BResNet_VAE%5D_Exp2_3_1_Transfer_Learning_Using_ResNet_VAE'Encoer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# README.MD

* For Variant Malware Classification Use pre-trained ResNet-VAE's Encoder Network
    * Load pre-trained ResNet-VAE's Encoder Network
        * Model State dic and Optim State dic
    * 1. Pass Malimg Data to Encoder Network --> Exp 01
    * 2. Train transfered Encoder for Malware Classification --> Exp 02

✅ Check Point
>  1. Load ResNet-VAE's 92th Model&Optim Parameters
>  2. Hyperparameter Optimization
>  3. Save Model's Parameter
>  4. Exp Report & Save Results

<code>from--- 20.09.28.mon am 00:30 ~ </code>







---
 ✋ Hyperparameter Optimization(하이퍼 파라미터 최적화)는 조금 미뤄두자.<br>

우선은 계획대로 실험을 진행해보자...! -20.09.28.mon-
---

# Mount Drive



In [1]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


<code>Auto Reload</code>

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%cd drive/My\ Drive/Post_InfoSec_Exps/ResNet-VAE/ResNetVAE-master.zip (Unzipped Files)/ResNetVAE-master
# ! ls

/content/drive/My Drive/Post_InfoSec_Exps/ResNet-VAE/ResNetVAE-master.zip (Unzipped Files)/ResNetVAE-master


In [4]:
import os 

def check_mkdir(dir_name):
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)

In [5]:
check_mkdir('./ResNet-VAE_Exp2_3_1.ResNet-VAE_Encoder_Transfer_Learning')

In [6]:
! ls

'01.Tutorial-ResNet-VAE.ipynb의 사본'
 01.Tutorial-ResNet-VAE-Recon.ipynb
 02.Tutorial-ResNet-VAE-Tunning.ipynb
 03.Tutorial-ResNet-VAE-Tunning.ipynb
'03.Tutorial-ResNet-VAE-Tunning.ipynb의 사본'
'04.Post-01.Tutorial-ResNet-VAE.ipynb사본의 사본'
'05.Post-01.Tutorial-ResNet-VAE_Train_Again.ipynb의 사본'
 19train_val_plot.png
 39train_val_plot.png
 59train_val_plot.png
 Again_ResNet-VAE_Exp01
'[Again][ResNet-VAE] Exp2_3_1. Transfer Learning Using ResNet-VAE'\''Encoer.ipynb'
'[Again] ResNet-VAE_plus_Classifier .ipynb'
'[Again] ResNet-VAE_plus_Classifier .ipynb의 사본'
'[Again] Re-Start_Training_ResNet-VAE .ipynb'
 fig
 generated_Malimg.png
 modules.py
 plot_latent.ipynb
'[Post_Exp]04-2.ResNet-VAE_Train_Again.ipynb'
'[Post_Exp]04-2.ResNet-VAE_Train_Again.ipynb의 사본'
'[Post_Exp]04-3.ResNet-VAE_Train_Again.ipynb'
'[Post_Exp]04-3.ResNet-VAE_Train_Again.ipynb의 사본'
'[Post_Exp]05-2.ResNet-VAE_Reduce_lr.ipynb'
 __pycache__
 README.md
 reconstruction_Malimg.png
 ResNetVAE_cifar10.py
 

In [7]:
save_path =  'ResNet-VAE_Exp2_3_1.ResNet-VAE_Encoder_Transfer_Learning'
save_path  # 실험 결과 저장 경로 설정

'ResNet-VAE_Exp2_3_1.ResNet-VAE_Encoder_Transfer_Learning'

# #01. Import Library

In [8]:
from PIL import Image
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl

import seaborn as sns

# save single numpy array
from tempfile import TemporaryFile
from sklearn.manifold import TSNE

import torch
import torch.utils.data  # torch.utils.data

from torch import nn, optim
from torch.nn import functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd

# argparse
import argparse
import time

import torchvision 
from torchvision import datasets, transforms
from torchvision.utils import save_image

# load modules
from torchvision import models
from modules import *

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# #02. Data Preparation

In [9]:
transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3), # make 3-channel Images
    transforms.ToTensor()])  # Composes several transforms together.

# make custom dataset
trainset = torchvision.datasets.ImageFolder(root='../../../../InformationSecurity_Summer/malimg',
                                            transform=transforms)

In [10]:
classes = trainset.classes
classes

['Adialer.C',
 'Agent.FYI',
 'Allaple.A',
 'Allaple.L',
 'Alueron.gen!J',
 'Autorun.K',
 'C2LOP.P',
 'C2LOP.gen!g',
 'Dialplatform.B',
 'Dontovo.A',
 'Fakerean',
 'Instantaccess',
 'Lolyda.AA1',
 'Lolyda.AA2',
 'Lolyda.AA3',
 'Lolyda.AT',
 'Malex.gen!J',
 'Obfuscator.AD',
 'Rbot!gen',
 'Skintrim.N',
 'Swizzor.gen!E',
 'Swizzor.gen!I',
 'VB.AT',
 'Wintrim.BX',
 'Yuner.A']

* Make Custom Dataset

In [11]:
full_dataset = trainset

# maek train, val, test: 8:1:1
train_size = int(0.8 * len(full_dataset))
val_size = int(0.1 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size
print(train_size, val_size, test_size)

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size, test_size])

7471 933 935


In [12]:
import argparse

# ====== Random Seed Initialization ====== #
seed = 123
np.random.seed(seed)  # Reseed a legacy MT19937 BitGenerator
torch.manual_seed(seed)

parser = argparse.ArgumentParser()
args = parser.parse_args("")

# ===== Model ====== #
args.CNN_fc_hidden1 = 1024
args.CNN_fc_hidden2 = 1024
args.CNN_embed_dim = 256  # latent dim extracted by 2D CNN
args.res_size = 224       # ResNet Image size

# ===== Regularization ===== #
args.dropout_p = 0.2           # dropout probability

# ===== training parameters ====== #
args.epochs = 20
args.learning_rate = 1e-3

args.batch_size = 50
args.log_interval = 10  # interval for displaying training info

print(args)

"""
# ===== Experiment Variable ====== #
# Reference: https://github.com/Steve-YJ/Assignment_Standalone_DL/blob/master/%5BRe_Fact%5D%20CIFAR-10_CNN_Report_Result.ipynb

name_var1 = 'lr'
name_var2 = 'hiden_layer_num'
list_var1 = [1e-3, 1e-4, 1e-5]
list_var2 = [1024, 512, 256, 128, 64, 32]


for var1 in list_var1:
    for var2 in list_var2:
        # setattr ??: name_var1('n_layer')를 1, 2, 3으로 바꿔준다
        # setattr = args.name_var1 = var1
        setattr(args, name_var1, var1)
        setattr(args, name_var2, var2)
        print(args)
                
        setting, result = experiment(, deepcopy(args))
        save_exp_result(setting, result)
"""

Namespace(CNN_embed_dim=256, CNN_fc_hidden1=1024, CNN_fc_hidden2=1024, batch_size=50, dropout_p=0.2, epochs=20, learning_rate=0.001, log_interval=10, res_size=224)


"\n# ===== Experiment Variable ====== #\n# Reference: https://github.com/Steve-YJ/Assignment_Standalone_DL/blob/master/%5BRe_Fact%5D%20CIFAR-10_CNN_Report_Result.ipynb\n\nname_var1 = 'lr'\nname_var2 = 'hiden_layer_num'\nlist_var1 = [1e-3, 1e-4, 1e-5]\nlist_var2 = [1024, 512, 256, 128, 64, 32]\n\n\nfor var1 in list_var1:\n    for var2 in list_var2:\n        # setattr ??: name_var1('n_layer')를 1, 2, 3으로 바꿔준다\n        # setattr = args.name_var1 = var1\n        setattr(args, name_var1, var1)\n        setattr(args, name_var2, var2)\n        print(args)\n                \n        setting, result = experiment(, deepcopy(args))\n        save_exp_result(setting, result)\n"

* z_loader's batch_size: 9339 -> 512

In [13]:
# make custom data_loader
train_loader = DataLoader(train_dataset,
                         batch_size=args.batch_size,
                         shuffle=True,
                         pin_memory=True)
val_loader = DataLoader(val_dataset,
                        batch_size=args.batch_size,
                        shuffle=True,
                        pin_memory=True)
 
test_loader = DataLoader(test_dataset,
                        batch_size=args.batch_size,
                        shuffle=True,
                        pin_memory=True)  # Instead, we recommend using automatic memory pinning (i.e., setting pin_memory=True)
                                          #  which enables fast data transfer to CUDA-enabled GPUs

# First, insert all test dataset
# z_loader: for latent vector extraction
z_loader = DataLoader(full_dataset,
                        batch_size=32,  # 128 -> 64 -> 32
                        shuffle=False,  # Shuffle -> False
                        pin_memory=True)

# #03. Model Architecture
## Make ResNet-VAE Architecture
* Referenced: https://github.com/hsinyilin19/ResNetVAE/blob/master/modules.py



실험 결과 저장 경로 설정 <code>save_model_path</code>

In [14]:
save_model_path = save_path
save_model_path

'ResNet-VAE_Exp2_3_1.ResNet-VAE_Encoder_Transfer_Learning'

In [15]:
def loss_function(recon_x, x, mu, logvar):
    # MSE = F.mse_loss(recon_x, x, reduction='sum')
    MSE = F.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return MSE + KLD

In [16]:
class ResNet_VAE(nn.Module):
    def __init__(self, fc_hidden1=1024, fc_hidden2=768, drop_p=0.3, CNN_embed_dim=256):
        super(ResNet_VAE, self).__init__()

        self.fc_hidden1, self.fc_hidden2, self.CNN_embed_dim = fc_hidden1, fc_hidden2, CNN_embed_dim

        # CNN architechtures
        self.ch1, self.ch2, self.ch3, self.ch4 = 16, 32, 64, 128
        self.k1, self.k2, self.k3, self.k4 = (5, 5), (3, 3), (3, 3), (3, 3)      # 2d kernal size
        self.s1, self.s2, self.s3, self.s4 = (2, 2), (2, 2), (2, 2), (2, 2)      # 2d strides
        self.pd1, self.pd2, self.pd3, self.pd4 = (0, 0), (0, 0), (0, 0), (0, 0)  # 2d padding

        # ====== Encoder ====== #
        # encoding components
        resnet = models.resnet152(pretrained=True)
        modules = list(resnet.children())[:-1]      # delete the last fc layer.
                                                    # pre-trained ResNet Architecture에서 마지막 FC레이어를 제외한 Model
        # ====== Notice ======= #
        # 이 코드를 잘 고치면 MLP Layer의 개수를 변경해 가면서 실험할 수 있겠다
        self.resnet = nn.Sequential(*modules)
        self.fc1 = nn.Linear(resnet.fc.in_features, self.fc_hidden1)
        self.bn1 = nn.BatchNorm1d(self.fc_hidden1, momentum=0.01)
        self.fc2 = nn.Linear(self.fc_hidden1, self.fc_hidden2)
        self.bn2 = nn.BatchNorm1d(self.fc_hidden2, momentum=0.01)

        # Latent vectors mu and sigma
        self.fc3_mu = nn.Linear(self.fc_hidden2, self.CNN_embed_dim)      # output = CNN embedding latent variables
        self.fc3_logvar = nn.Linear(self.fc_hidden2, self.CNN_embed_dim)  # output = CNN embedding latent variables

        # Sampling vector
        self.fc4 = nn.Linear(self.CNN_embed_dim, self.fc_hidden2)
        self.fc_bn4 = nn.BatchNorm1d(self.fc_hidden2)
        self.fc5 = nn.Linear(self.fc_hidden2, 64 * 4 * 4)
        self.fc_bn5 = nn.BatchNorm1d(64 * 4 * 4)
        self.relu = nn.ReLU(inplace=True)

        # ====== Decoder ====== #
        # Decoder
        self.convTrans6 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=64, out_channels=32, kernel_size=self.k4, stride=self.s4,
                               padding=self.pd4),
            nn.BatchNorm2d(32, momentum=0.01),
            nn.ReLU(inplace=True),
        )
        self.convTrans7 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=32, out_channels=8, kernel_size=self.k3, stride=self.s3,
                               padding=self.pd3),
            nn.BatchNorm2d(8, momentum=0.01),
            nn.ReLU(inplace=True),
        )

        self.convTrans8 = nn.Sequential(
            nn.ConvTranspose2d(in_channels=8, out_channels=3, kernel_size=self.k2, stride=self.s2,
                               padding=self.pd2),
            nn.BatchNorm2d(3, momentum=0.01),
            nn.Sigmoid()    # y = (y1, y2, y3) \in [0 ,1]^3
        )


    def encode(self, x):
        x = self.resnet(x)  # ResNet
        x = x.view(x.size(0), -1)  # flatten output of conv

        # FC layers
        x = self.bn1(self.fc1(x))
        x = self.relu(x)
        x = self.bn2(self.fc2(x))
        x = self.relu(x)
        # x = F.dropout(x, p=self.drop_p, training=self.training)
        mu, logvar = self.fc3_mu(x), self.fc3_logvar(x)
        return mu, logvar  # check: Encoder -> hid1-BN-hid2-BN ==> x

    def reparameterize(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = Variable(std.data.new(std.size()).normal_())
            return eps.mul(std).add_(mu)
        else:
            return mu

    def decode(self, z):
        x = self.relu(self.fc_bn4(self.fc4(z)))
        x = self.relu(self.fc_bn5(self.fc5(x))).view(-1, 64, 4, 4)
        x = self.convTrans6(x)
        x = self.convTrans7(x)
        x = self.convTrans8(x)
        x = F.interpolate(x, size=(224, 224), mode='bilinear')
        return x

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        x_reconst = self.decode(z)

        return x_reconst, z, mu, logvar

📌 Custom DataLoader의 경우, 자동화를 위해 Train, Val, Test 함수 내에 만들어준다

In [17]:
# Detect devices
use_cuda = torch.cuda.is_available()
print(use_cuda)
device = torch.device("cuda:0" if use_cuda else "cpu")
print(device)

True
cuda:0


In [ ]:
print(epoch)

90


95 Epoch이 최선이지만... 데이터를 지워버렸기에 ㅠ 차선인 Epoch 90의 값을 불러온다...!

아래의 셀을 실행하면 ./results_ResNet-VAE_Exp01에 저장되어있는 pre-trained model의 parameter값을 불러올 수 있다

In [18]:
### If you want to use pre-trained model ####
pre_saved_model_path = './results_ResNet-VAE_Exp01'
epoch=90

# Create model
resnet_vae = ResNet_VAE(fc_hidden1=args.CNN_fc_hidden1, fc_hidden2=args.CNN_fc_hidden2, drop_p=args.dropout_p, CNN_embed_dim=args.CNN_embed_dim).to(device)
### If you want to use pre-trained model ####
resnet_vae.load_state_dict(torch.load(os.path.join(pre_saved_model_path, 'model_epoch{}.pth'.format(epoch))))

model_params = list(resnet_vae.parameters())
optimizer = torch.optim.Adam(model_params, lr=args.learning_rate)
### If you want to use pre-trained model's optimizer ####
optimizer.load_state_dict(torch.load(os.path.join(pre_saved_model_path, 'optimizer_epoch{}.pth'.format(epoch))))


print('Number of {} parameters'.format(sum(p.numel() for p in resnet_vae.parameters() if p.requires_grad)))
print("Using", torch.cuda.device_count(), "GPU!")

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /root/.cache/torch/hub/checkpoints/resnet152-b121ed2d.pth



Number of 63158425 parameters
Using 1 GPU!


In [19]:
resnet_vae

ResNet_VAE(
  (resnet): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(64

# #04. Train, Validation, Test
After train ResNet-VAE, Use EncoderNet as Feature Extractor<br>
<br>
* Extract Latent Vector
* Improve Classifier

* pre-traned ResNet-VAE의 Encoder를 <code>Feature Extractor</code>로 활용하여 분류 문제를 풀어보도록 한다. <code>20.09.28.mon pm4:00</code>

* Referernce:
* 추가실험 List
    * Load VAE_State_Dict
    * Extract Latent vector & make Training set(for DL/ML)
    * => For all training data: (9339, 256)  / Latent dim
    * => Add Labels: (9339, 101)
    * Make Pandas DataFrame
    * Make pd.DataFrame()
    * Save DataFrame() to csv
    * Make Classifier
* Classification
    * ML: Voting Classifier
    * DL: ...

# ResNet-VAE & Ensemble 

## Extract Latent Vector

* 적당한 알고리즘을 짜야겠구나...! -20.09.28.mon pm6:50-

In [20]:
for i, (data, _) in enumerate(z_loader):  # load_whole data(9339)
    data = data.to(device)
    # recon_batch, mu, logvar, z = resnet_vae(data)
    recon_batch, z, mu, logvar = resnet_vae(data)  # batch_size만큼 값 저장
    
    # save latent_vector
    latent_vector = z.detach().cpu().clone().numpy()  # change tensor type data to cpu().numpy()
                                                        # latent_vector_size: (num_of_data, 256)  # num_of_data, num_of_dim
    # save label
    label = _.detach().cpu().clone().numpy()
    label =label.reshape(1, -1)
    print(label.shape)

    # concat_data
    if i == 0:
        latent_z = np.concatenate((latent_vector, label.T), axis=1)  # latent_z: (9339, 101): (num_data, latent_z + label) 
        print(latent_z.shape)  # (32, 257): batch_num, features + label
    elif i > 0:
        latent_z = np.concatenate((latent_z,  np.concatenate((latent_vector, label.T), axis=1)), axis=0)
        print(latent_z.shape)


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3121: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


(1, 32)
(32, 257)
(1, 32)
(64, 257)
(1, 32)
(96, 257)
(1, 32)
(128, 257)
(1, 32)
(160, 257)
(1, 32)
(192, 257)
(1, 32)
(224, 257)
(1, 32)
(256, 257)
(1, 32)
(288, 257)
(1, 32)
(320, 257)
(1, 32)
(352, 257)
(1, 32)
(384, 257)
(1, 32)
(416, 257)
(1, 32)
(448, 257)
(1, 32)
(480, 257)
(1, 32)
(512, 257)
(1, 32)
(544, 257)
(1, 32)
(576, 257)
(1, 32)
(608, 257)
(1, 32)
(640, 257)
(1, 32)
(672, 257)
(1, 32)
(704, 257)
(1, 32)
(736, 257)
(1, 32)
(768, 257)
(1, 32)
(800, 257)
(1, 32)
(832, 257)
(1, 32)
(864, 257)
(1, 32)
(896, 257)
(1, 32)
(928, 257)
(1, 32)
(960, 257)
(1, 32)
(992, 257)
(1, 32)
(1024, 257)
(1, 32)
(1056, 257)
(1, 32)
(1088, 257)
(1, 32)
(1120, 257)
(1, 32)
(1152, 257)
(1, 32)
(1184, 257)
(1, 32)
(1216, 257)
(1, 32)
(1248, 257)
(1, 32)
(1280, 257)
(1, 32)
(1312, 257)
(1, 32)
(1344, 257)
(1, 32)
(1376, 257)
(1, 32)
(1408, 257)
(1, 32)
(1440, 257)
(1, 32)
(1472, 257)
(1, 32)
(1504, 257)
(1, 32)
(1536, 257)
(1, 32)
(1568, 257)
(1, 32)
(1600, 257)
(1, 32)
(1632, 257)
(1, 32)
(1664,

In [21]:
latent_z.shape

(9339, 257)

save_model_path = 'ResNet-VAE_Exp2_3_1.ResNet-VAE_Encoder_Transfer_Learning'

In [29]:
with open(('./'+save_model_path+'/{}Epoch_z_vector.npy'.format(epoch)), 'wb') as f:
        np.save(f, latent_z)  # but latent vector size is (16, 100).... just 16...
                                    # 1 Epoch 단위로 latent vector를 저장한다
                                    # 이 떄, latent vector의 size는 test_dataset의 크기가 되어야 한다
                                    # 결과값이 좋은 vector는 좋은 feature로 사용할 수 있다.
# plot latent vector Every 10 Epochs
print("Svae Latent vector!")

Svae Latent vector!


In [23]:
latent_z.shape

(9339, 257)

## Data Preparation

In [30]:
latent_vector = np.load('./'+save_model_path+'/{}Epoch_z_vector.npy'.format(epoch))
print("Load Latent_Vector!!")
latent_vector.shape

Load Latent_Vector!!


(9339, 257)

In [25]:
data = latent_vector
print(data.shape)

(9339, 257)


In [26]:
# numpy to pandas

import pandas as pd

data = pd.DataFrame(data=data)
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256
0,1.240568,-0.561261,-1.473960,-0.404962,0.245697,1.038918,1.497949,-0.449939,1.170961,-0.546789,-0.780814,1.361100,-1.116053,1.910452,1.412538,0.478924,0.373539,-2.539976,1.891036,0.528984,-1.579915,1.082758,0.560673,0.480609,0.039558,0.151825,-0.786566,-0.029136,1.107318,0.188695,0.495078,-2.756053,-0.256838,-1.180869,-0.122568,-0.018253,-1.009968,0.068781,0.692828,0.367560,...,0.567895,-1.819931,0.934984,-1.330302,-0.783299,0.377807,0.361624,1.070406,1.513979,1.717499,-1.591013,0.140981,0.141379,1.136888,1.406761,-0.208655,1.209202,2.338400,2.392874,2.012412,-0.083929,-0.493434,1.134609,1.298404,0.937509,-1.001865,1.713861,0.247447,0.213539,-0.402759,-0.426054,0.811708,1.755919,-1.964023,0.227610,-0.768265,-1.295807,-0.405288,0.462285,0.0
1,-1.996075,0.650010,-0.192071,-0.454621,-1.276196,-0.767499,-0.760912,0.700617,-0.618626,-0.309001,0.553122,-0.545304,-0.615293,-0.224021,-0.132841,2.032287,-0.279049,-0.337486,-0.510281,0.435519,-0.275984,0.629615,-1.974528,-0.119307,0.051937,1.114540,0.164383,0.793738,0.474289,0.513670,-0.203033,0.933196,0.038842,-0.398890,-1.007496,-2.026744,-0.937173,-0.139504,-2.569224,1.689294,...,-0.106298,-1.845786,1.896894,0.181780,-0.117664,0.027586,1.873160,0.400704,0.385745,1.251343,0.150390,0.823379,1.523866,0.766172,-0.780572,0.003207,-0.167644,-1.065065,1.136096,-1.246378,-0.335723,-0.483589,1.081218,-0.601342,0.948993,-0.955097,0.871614,-0.092773,0.880835,-1.132668,2.680984,2.019951,-1.422524,1.331898,-1.140199,-0.439111,1.876634,1.189979,1.602775,0.0
2,0.171825,0.165917,-0.203169,0.767052,0.591684,-1.322803,-0.564557,-1.029487,1.375214,0.408249,0.619360,1.183983,-0.888452,1.377965,0.774443,-0.445341,-1.066941,-0.125578,-1.220309,0.132049,-0.187190,-1.152096,0.454156,0.557448,-0.402943,-1.225979,0.127531,0.069677,-0.021814,-0.184043,0.218814,0.811766,0.761276,-1.585932,-1.304321,0.297011,0.376011,0.308563,0.393351,-0.975130,...,-0.994814,0.706768,0.388398,0.682229,0.553078,0.006548,0.635607,-0.107498,0.240612,0.177998,0.529921,-1.271759,0.508364,-0.844720,0.723552,1.203746,-0.178693,-1.185009,-1.225258,-0.080642,1.270459,0.549151,-0.083043,0.940601,-0.525292,-0.501228,1.030271,-0.824522,1.334434,-1.053619,-0.267697,-1.669339,-0.214436,0.967534,0.748140,0.714027,-0.638480,-0.124279,0.366332,0.0
3,0.418800,-0.323658,-0.353201,0.025223,0.577787,-0.317023,-0.554338,0.847408,-0.778099,0.850690,1.304112,-0.434819,-0.771734,-0.899485,-1.215115,-0.775774,-0.647416,-0.542243,-0.782168,1.324322,0.349843,-0.937292,0.429282,-1.328041,-1.736635,-1.935650,-0.518941,1.616143,1.371421,-0.277386,-0.162207,-0.983132,-0.485843,-0.145070,0.565974,0.353951,0.936010,0.721208,-0.076706,-1.273901,...,-0.363823,1.495660,0.103310,-1.316497,0.356881,0.567666,-0.200409,-0.110373,-0.781076,0.307884,-0.069431,-0.132790,-1.144245,0.333528,0.523856,0.867204,-0.016039,-1.615564,-1.395351,0.766038,1.031234,1.919342,-0.098541,-0.379554,0.981259,-0.855776,0.859527,0.458385,0.499763,0.061307,-0.604850,-1.219235,-0.852428,0.065624,1.236948,0.441680,-0.749305,-0.381263,-0.214752,0.0
4,0.892403,-1.174784,1.358834,0.672741,-0.919037,0.451660,1.270008,0.863308,0.221963,0.234472,0.736504,-0.086315,-1.201007,0.158627,-1.076775,1.030661,-0.773292,0.787029,-0.436171,-1.774708,1.092351,2.226049,-0.264831,0.176730,-3.456142,-1.355500,0.387206,0.151994,-2.672254,0.742959,-0.655962,-0.843967,-1.155222,0.157424,-0.010366,1.309886,-0.681162,2.305023,-0.637423,0.178157,...,0.439072,-1.782575,-1.465596,2.058444,-0.182285,-0.186349,0.868791,-0.181700,-0.072199,-0.986495,0.999168,-0.285254,0.090523,-1.240313,-1.137443,-0.208913,-1.043211,0.586708,-0.749179,-0.122272,-1.871484,0.067748,0.122957,0.033766,-1.158714,0.466875,-0.003578,-0.511014,0.396934,0.508805,-0.936099,-1.209

In [31]:
data.to_csv('./'+save_model_path+ '/ResNet-VAE_Latent_dataset.csv', index=False)
df = pd.read_csv('./'+save_model_path+ '/ResNet-VAE_Latent_dataset.csv')

In [32]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256
0,1.240568,-0.561261,-1.473960,-0.404962,0.245697,1.038918,1.497949,-0.449939,1.170961,-0.546789,-0.780814,1.361100,-1.116053,1.910452,1.412538,0.478924,0.373539,-2.539976,1.891036,0.528984,-1.579915,1.082758,0.560673,0.480609,0.039558,0.151825,-0.786566,-0.029136,1.107318,0.188695,0.495078,-2.756053,-0.256838,-1.180869,-0.122568,-0.018253,-1.009968,0.068781,0.692828,0.367560,...,0.567895,-1.819931,0.934984,-1.330302,-0.783299,0.377807,0.361624,1.070406,1.513979,1.717499,-1.591013,0.140981,0.141379,1.136888,1.406761,-0.208655,1.209202,2.338400,2.392874,2.012412,-0.083929,-0.493434,1.134609,1.298404,0.937509,-1.001865,1.713861,0.247447,0.213539,-0.402759,-0.426054,0.811708,1.755919,-1.964023,0.227610,-0.768265,-1.295807,-0.405288,0.462285,0.0
1,-1.996075,0.650010,-0.192071,-0.454621,-1.276196,-0.767499,-0.760912,0.700617,-0.618626,-0.309001,0.553122,-0.545304,-0.615293,-0.224021,-0.132841,2.032287,-0.279049,-0.337486,-0.510281,0.435519,-0.275984,0.629615,-1.974528,-0.119307,0.051937,1.114540,0.164383,0.793738,0.474289,0.513670,-0.203033,0.933196,0.038842,-0.398890,-1.007496,-2.026744,-0.937173,-0.139504,-2.569224,1.689294,...,-0.106298,-1.845786,1.896894,0.181780,-0.117664,0.027586,1.873160,0.400704,0.385745,1.251343,0.150390,0.823379,1.523866,0.766172,-0.780572,0.003207,-0.167644,-1.065065,1.136096,-1.246378,-0.335723,-0.483589,1.081218,-0.601342,0.948993,-0.955097,0.871614,-0.092773,0.880835,-1.132668,2.680984,2.019951,-1.422524,1.331898,-1.140199,-0.439111,1.876634,1.189979,1.602775,0.0
2,0.171825,0.165917,-0.203169,0.767052,0.591684,-1.322803,-0.564557,-1.029487,1.375214,0.408249,0.619360,1.183983,-0.888452,1.377965,0.774443,-0.445341,-1.066941,-0.125578,-1.220309,0.132049,-0.187190,-1.152096,0.454156,0.557448,-0.402943,-1.225979,0.127531,0.069677,-0.021814,-0.184043,0.218814,0.811766,0.761276,-1.585932,-1.304321,0.297011,0.376011,0.308563,0.393351,-0.975130,...,-0.994814,0.706768,0.388398,0.682229,0.553078,0.006548,0.635607,-0.107498,0.240612,0.177998,0.529921,-1.271759,0.508364,-0.844720,0.723552,1.203746,-0.178693,-1.185009,-1.225258,-0.080642,1.270459,0.549151,-0.083043,0.940601,-0.525292,-0.501228,1.030271,-0.824522,1.334434,-1.053619,-0.267697,-1.669339,-0.214436,0.967534,0.748140,0.714027,-0.638480,-0.124279,0.366332,0.0
3,0.418800,-0.323658,-0.353201,0.025223,0.577787,-0.317023,-0.554338,0.847408,-0.778099,0.850690,1.304112,-0.434819,-0.771734,-0.899485,-1.215115,-0.775774,-0.647416,-0.542243,-0.782168,1.324322,0.349843,-0.937292,0.429282,-1.328041,-1.736635,-1.935650,-0.518941,1.616143,1.371421,-0.277386,-0.162207,-0.983132,-0.485843,-0.145070,0.565974,0.353951,0.936010,0.721208,-0.076706,-1.273901,...,-0.363823,1.495660,0.103310,-1.316497,0.356881,0.567666,-0.200409,-0.110373,-0.781076,0.307884,-0.069431,-0.132790,-1.144245,0.333528,0.523856,0.867204,-0.016039,-1.615564,-1.395351,0.766038,1.031234,1.919342,-0.098541,-0.379554,0.981259,-0.855776,0.859527,0.458385,0.499763,0.061307,-0.604850,-1.219235,-0.852428,0.065624,1.236948,0.441680,-0.749305,-0.381263,-0.214752,0.0
4,0.892403,-1.174784,1.358834,0.672741,-0.919037,0.451660,1.270008,0.863308,0.221963,0.234472,0.736504,-0.086315,-1.201007,0.158627,-1.076775,1.030661,-0.773292,0.787029,-0.436171,-1.774708,1.092351,2.226049,-0.264831,0.176730,-3.456142,-1.355500,0.387206,0.151994,-2.672254,0.742959,-0.655962,-0.843967,-1.155222,0.157424,-0.010366,1.309886,-0.681162,2.305023,-0.637423,0.178157,...,0.439072,-1.782575,-1.465596,2.058444,-0.182285,-0.186349,0.868791,-0.181700,-0.072199,-0.986495,0.999168,-0.285254,0.090523,-1.240313,-1.137443,-0.208913,-1.043211,0.586708,-0.749179,-0.122272,-1.871484,0.067748,0.122957,0.033766,-1.158714,0.466875,-0.003578,-0.511014,0.396934,0.508805,-0.936099,-1.209

## Ensemble Classifier
### 01. Voting Classifier

In [34]:
# Test-Voting classifier
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html


from sklearn import datasets
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

# iris = datasets.load_iris()
data = latent_vector  # re-change it to numpy
X, y = data[:, :256], data[:, 256]

clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(n_estimators=100, random_state=1)
clf3 = GaussianNB()

eclf = VotingClassifier(
         estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)],
         voting='hard')

for clf, label in zip([clf1, clf2, clf3, eclf], ['Logistic Regression', 'Random Forest', 'naive Bayes', 'Ensemble']):
     scores = cross_val_score(clf, X, y, scoring='accuracy', cv=10)  # cross validation = 10
     print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Accuracy: 0.31 (+/- 0.01) [Logistic Regression]
Accuracy: 0.62 (+/- 0.03) [Random Forest]
Accuracy: 0.44 (+/- 0.01) [naive Bayes]


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Accuracy: 0.51 (+/- 0.02) [Ensemble]


In [35]:
print("Done!")

Done!


In [37]:
# Test-Voting classifier
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html


from sklearn import datasets
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

# iris = datasets.load_iris()
data = latent_vector  # re-change it to numpy
X, y = data[:, :256], data[:, 256]
print('='*20,'Shape of X and y', '='*20)
print(X.shape, y.shape)
print('='*50)


clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(n_estimators=50, random_state=1)
clf3 = GaussianNB()

eclf = VotingClassifier(
         estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)],
         voting='hard')

for clf, label in zip([clf1, clf2, clf3, eclf], ['Logistic Regression', 'Random Forest', 'naive Bayes', 'Ensemble']):
     scores = cross_val_score(clf, X, y, scoring='accuracy', cv=10)  # cross validation = 10
     print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))
print("Done!!")

==================== Shape of X and y ====================
(9339, 256) (9339,)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Accuracy: 0.31 (+/- 0.01) [Logistic Regression]
Accuracy: 0.61 (+/- 0.02) [Random Forest]
Accuracy: 0.44 (+/- 0.01) [naive Bayes]


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Accuracy: 0.51 (+/- 0.02) [Ensemble]
Done!!


### Voting Classifier3
* 5 RandomForest


In [39]:
# Test-Voting classifier
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html


from sklearn import datasets
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

# iris = datasets.load_iris()
data = latent_vector  # re-change it to numpy
X, y = data[:, :256], data[:, 256]
print('='*20,'Shape of X and y', '='*20)
print(X.shape, y.shape)
print('='*50)


clf1 = RandomForestClassifier(n_estimators=50, random_state=1)
clf2 = RandomForestClassifier(n_estimators=50, random_state=1)
clf3 = RandomForestClassifier(n_estimators=50, random_state=1)
clf4 = RandomForestClassifier(n_estimators=50, random_state=1)
clf5 = RandomForestClassifier(n_estimators=50, random_state=1)

eclf = VotingClassifier(
         estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)],
         voting='hard')

for clf, label in zip([clf1, clf2, clf3, clf4, clf5,  eclf], ['Random Forest', 'Random Forest', 'Random Forest', 'Random Forest', 'Random Forest', 'Ensemble']):
     scores = cross_val_score(clf, X, y, scoring='accuracy', cv=10)  # cross validation = 10
     print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))
print("Done!!")

==================== Shape of X and y ====================
(9339, 256) (9339,)
Accuracy: 0.61 (+/- 0.02) [Random Forest]
Accuracy: 0.61 (+/- 0.02) [Random Forest]
Accuracy: 0.61 (+/- 0.02) [Random Forest]
Accuracy: 0.61 (+/- 0.02) [Random Forest]
Accuracy: 0.61 (+/- 0.02) [Random Forest]
Accuracy: 0.61 (+/- 0.02) [Ensemble]
Done!!


📌 최근에 Refactoring한 코드를 참고해서 본 연구의 실험 코드도 Refactoring 해보자..! <code>-20.09.28.Mon am 01:00</code>

### Voting Classifier4
* 👨‍🏫 1.11.6.4. Using the VotingClassifier with GridSearchCV
   * GridSearch를 통해 최적의 파라미터를 찾아보자
   * sklearn Documentation 참고

Voting Classifier can also be used together with GridSearchCV in order to tune the hyperparameters of the individual estimators

In [ ]:
from sklearn import datasets
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

# iris = datasets.load_iris()
data = latent_vector  # re-change it to numpy
X, y = data[:, :256], data[:, 256]
print('='*20,'Shape of X and y', '='*20)
print(X.shape, y.shape)
print('='*50)


clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(random_state=7)
clf3 = GaussianNB()
eclf = VotingClassifier(
    estimators = [('lr', clf1), ('rf', clf2), ('gnb', clf3)],
    voting='soft'
)

params = {'lr__C': [1.0, 100.0], 'rf__n_estimators': [20, 200]}

grid = GridSearchCV(estimator=eclf, param_grid=params, cv=10)
grid = grid.fit(X, y)

==================== Shape of X and y ====================
(9339, 256) (9339,)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [ ]:
for clf, label in zip([clf1, clf2, clf3, eclf], ['Logistic Regression', 'Random Forest', 'naive Bayes', 'Ensemble']):
    scores = cross_val_score(clf, X, y, scoring='accuracy', cv=10)
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

아래의 train code는 ResNet-VAE train Code이다<br>
📌📌 => Encoder Network의 Train Code로 변경시켜보자...!<br>
* 참고하기
    * Reference: https://github.com/Steve-YJ/Assignment_Standalone_DL/blob/master/%5BRe_Fact%5D%20CIFAR-10_CNN_Report_Result.ipynb

* Encoder Network에서 <code>mu와 logvar</code>을 잘 활용해보자 -20.09.28.mon-
    * ResNet-152 -> FC1 -> FC2 -> mu or logvar

## Original Train Code - for ResNet-VAE Training

In [ ]:
def train(log_interval, model, device, train_loader, optimizer, epoch):
    # set model as training mode
    model.train()

    losses = []
    all_y, all_z, all_mu, all_logvar = [], [], [], []
    N_count = 0   # counting total trained sample in one epoch
    for batch_idx, (X, y) in enumerate(train_loader):
        # distribute data to device
        X, y = X.to(device), y.to(device).view(-1, )
        N_count += X.size(0)  # count batch_size sample

        optimizer.zero_grad()
        X_reconst, z, mu, logvar = model(X)  # VAE
        loss = loss_function(X_reconst, X, mu, logvar)
        losses.append(loss.item())

        loss.backward()
        optimizer.step()

        all_y.extend(y.data.cpu().numpy())
        all_z.extend(z.data.cpu().numpy())
        all_mu.extend(mu.data.cpu().numpy())
        all_logvar.extend(logvar.data.cpu().numpy())

        # show information
        if (batch_idx + 1) % log_interval == 0:  # if batch_size = 16 => 160
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch + 1, N_count, len(train_loader.dataset), 100. * (batch_idx + 1) / len(train_loader), loss.item()))

    # calculate train_loss
    losses /= len(train_loader.dataset)
    all_y = np.stack(all_y, axis=0)
    all_z = np.stack(all_z, axis=0)
    all_mu = np.stack(all_mu, axis=0)
    all_logvar = np.stack(all_logvar, axis=0)

    # save Pytorch models of best record
    torch.save(model.state_dict(), os.path.join(save_model_path, 'model_epoch{}.pth'.format(epoch + 1)))  # save motion_encoder
    torch.save(optimizer.state_dict(), os.path.join(save_model_path, 'optimizer_epoch{}.pth'.format(epoch + 1)))      # save optimizer
    print("Epoch {} model saved!".format(epoch + 1))

    return X.data.cpu().numpy(), all_y, all_z, all_mu, all_logvar, losses



## Train Function for EncoderNet

* <code>Refactoring</code> Train Function for EncoderNet

In [ ]:
def train(net, trainloader, optimizer, criterion, args):
    
    """
    이미 위에서 정의를 해줌
    trainloader = torch.utils.data.DataLoader(partition['train'], 
                                              batch_size=args.train_batch_size, 
                                              shuffle=True, num_workers=2)
    """

    net.train()  # train mode
    optimizer.zero_grad()  # optimizer 초기화

    correct = 0
    total = 0
    train_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        # print('input dimension: ', inputs.shape)
        # raise RuntimeError
        # inputs = inputs.view(-1, 3072)
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = net.encoder(inputs)
        

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = train_loss / len(trainloader)
    train_acc = 100 * correct / total
    return net, train_loss, train_acc

In [ ]:
def val(net, valloader, optimizer, criterion, args):
    
    """
    이미 위에서 정의를 해줌
    valloader = torch.utils.data.DataLoader(partition['train'], 
                                              batch_size=args.test_batch_size, 
                                              shuffle=True, num_workers=2)
    """

    net.eval()

    correct = 0
    total = 0
    val_loss = 0.0

    with torch.no_grad():
        for i, data in enumerate(valloader, 0):
            # get the inputs
            inputs, labels = data
            # print('input dimension: ', inputs.shape)
            # raise RuntimeError
            # inputs = inputs.view(-1, 3072)
            inputs = inputs.cuda()
            labels = labels.cuda()
            outputs = net(inputs)

            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        val_loss = val_loss / len(valloader)
        val_acc = 100 * correct / total
    return val_loss, val_acc

In [ ]:
def test(net, testloader, args):
    
    """
    이미 위에서 정의를 해줌
    testloader = torch.utils.data.DataLoader(partition['train'], 
                                              batch_size=args.test_batch_size, 
                                              shuffle=True, num_workers=2)
    """

    net.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for i, data in enumerate(testloader, 0):
            # get the inputs
            inputs, labels = data
            # print('input dimension: ', inputs.shape)
            # raise RuntimeError
            # inputs = inputs.view(-1, 3072)
            inputs = inputs.cuda()
            labels = labels.cuda()
            outputs = net(inputs)
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        test_acc = 100 * correct / total
    return test_acc

In [ ]:
def experiment(trainloader, valloader, testloader, args):

    """
    net = CNN(model_code = args.model_code,
              in_channels = args.in_channels,
              out_dim = args.out_dim,
              act = args.act,
              use_bn = args.use_bn)
    net.cuda()
    """
    # Detect devices
    use_cuda = torch.cuda.is_available()
    print(use_cuda)
    device = torch.device("cuda:0" if use_cuda else "cpu")
    print(device)

    """ 
    # ====== If you want to use pre-trained model ====== #
    """
    pre_saved_model_path = './results_ResNet-VAE_Exp01'
    epoch=90  # pre-trained ResNet-VAE parameter

    # Create model
    resnet_vae = ResNet_VAE(fc_hidden1=args.CNN_fc_hidden1, fc_hidden2=args.CNN_fc_hidden2, drop_p=args.dropout_p, CNN_embed_dim=args.CNN_embed_dim).to(device)
    ### If you want to use pre-trained model ####
    resnet_vae.load_state_dict(torch.load(os.path.join(pre_saved_model_path, 'model_epoch{}.pth'.format(epoch))))
    
    """
    # resnet_vae에서 encodernet을 어떻게 사용할 수 있을까? -20.09.28.mon-
    """
    


    model_params = list(resnet_vae.parameters())
    optimizer = torch.optim.Adam(model_params, lr=args.learning_rate)
    ### If you want to use pre-trained model's optimizer ####
    optimizer.load_state_dict(torch.load(os.path.join(pre_saved_model_path, 'optimizer_epoch{}.pth'.format(epoch))))


    print('Number of {} parameters'.format(sum(p.numel() for p in resnet_vae.parameters() if p.requires_grad)))
    print("Using", torch.cuda.device_count(), "GPU!")


    criterion = nn.CrossEntropyLoss()
    if args.optim == 'SGD':
        optimizer = optim.SGD(net.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'RMSprop':
        optimizer = optim.RMSprop(net.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'Adam':
        optimizer = optim.Adam(net.parameters(), lr=args.lr, weight_decay=args.l2)
    else:
        raise ValueError('In-valid optimizer choice')
    
    # ===== List for epoch-wise data ====== #
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []
    # ================= Training ==================== #
        
    for epoch in range(args.epoch):  # loop over the dataset multiple times
        ts = time.time()
        net, train_loss, train_acc = train(net, trainloader, optimizer, criterion, args)
        val_loss, val_acc = validate(net, valloader, criterion, args)
        te = time.time()
        
        # ====== Add Epoch Data ====== #
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accs.append(train_acc)
        val_accs.append(val_acc)
        # ============================ #
        
        print('Epoch {}, Acc(train/val): {:2.2f}/{:2.2f}, Loss(train/val) {:2.2f}/{:2.2f}. Took {:2.2f} sec'.format(epoch, train_acc, val_acc, train_loss, val_loss, te-ts))
        
    test_acc = test(net, testloader, args)    
    
    # ======= Add Result to Dictionary ======= #
    # 전체 결과값을 저장하는 Results Dic 
    # Q. 왜 때문인진 모르겠지만 train_accs와 train_acc가 분리되어있네?!
        # train_acc와 val_acc은 없어도 되는거 아냐?
    result = {}
    result['train_losses'] = train_losses
    result['val_losses'] = val_losses
    result['train_accs'] = train_accs
    result['val_accs'] = val_accs
    # result['train_acc'] = train_acc
    # result['val_acc'] = val_acc
    result['test_acc'] = test_acc
    return vars(args), result  # vars(args)를 해주면 args를 dic로 저장해준다
    # ===================================== #

# #05. Manage Experiment

In [ ]:
import hashlib
import json
from os import listdir
from os.path import isfile, join
import pandas as pd

def save_exp_result(setting, result):  # setting: args
                                       # result : train_losses, val_losses, train_accs, val_accs,  
    exp_name = setting['exp_name']
    del setting['epoch']               # setting에서 'epoch'과 'test_batch_size'를 제거해준다고...
    del setting['test_batch_size']     # 왜 떼는거지?

    hash_key = hashlib.sha1(str(setting).encode()).hexdigest()[:6]  # setting에 따라 다른 파일명을 갖도록 만들어준다
    filename = './'+save_model_path+'/{}-{}.json'.format(exp_name, hash_key)
    # print(filename)
    result.update(setting)  # 결과 dictionary에 setting값을 더해준다 => Dic
    with open(filename, 'w') as f:
        json.dump(result, f)  # JSON 포멧으로 dictionary 값 저장

    
def load_exp_result(exp_name):
    dir_path = './results'
    filenames = [f for f in listdir(dir_path) if isfile(join(dir_path, f)) if '.json' in f]  # ./resuts에 저장되어있는 file들을 리스트 형태로 저장
    list_result = []
    for filename in filenames:
        if exp_name in filename:
            # print(exp_name)
            with open(join(dir_path, filename), 'r') as infile:
                results = json.load(infile)
                list_result.append(results)  # Dic을 list형태로 저장
    df = pd.DataFrame(list_result) # .drop(columns=[])
    return df


# #06. Experiment


In [ ]:
# ====== Random Seed Initialization ====== #
seed = 123  # 랜덤 시드값 초기화
np.random.seed(seed)
torch.manual_seed(seed)

parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.exp_name = "exp1_lr_model_code"

# ====== Model ====== #
# args.model_code ='VGG11'
# args.in_channels = 3
args.out_dim = 10
# args.hid_dim = 100
args.act = 'relu'
args.CNN_fc_hidden1 = 1024
args.CNN_fc_hidden2 = 1024
args.CNN_embed_dim = 256  # latent dim extracted by 2D CNN
args.res_size = 224       # ResNet Image size
args.dropout_p = 0.2           # dropout probability



# ====== Regularization ======= #
# args.l2 = 0.00001
# args.use_bn = True

# ====== Optimizer & Training ====== #
args.epochs = 20
args.learning_rate = 1e-3

args.batch_size = 50
args.log_interval = 10  # interval for displaying training info

print(args)



# ===== Experiment Variable ====== #
# Reference: https://github.com/Steve-YJ/Assignment_Standalone_DL/blob/master/%5BRe_Fact%5D%20CIFAR-10_CNN_Report_Result.ipynb

name_var1 = 'lr'
name_var2 = 'hiden_layer_num'
list_var1 = [1e-3, 1e-4, 1e-5]
list_var2 = [1024, 512, 256, 128, 64, 32]


for var1 in list_var1:
    for var2 in list_var2:
        # setattr ??: name_var1('n_layer')를 1, 2, 3으로 바꿔준다
        # setattr = args.name_var1 = var1
        setattr(args, name_var1, var1)
        setattr(args, name_var2, var2)
        print(args)
                
        setting, result = experiment(train_loader, val_loader, test_loader, args)
        save_exp_result(setting, result)

Namespace(CNN_embed_dim=256, CNN_fc_hidden1=1024, CNN_fc_hidden2=1024, act='relu', batch_size=50, dropout_p=0.2, epochs=20, exp_name='exp1_lr_model_code', learning_rate=0.001, log_interval=10, out_dim=10, res_size=224)
Namespace(CNN_embed_dim=256, CNN_fc_hidden1=1024, CNN_fc_hidden2=1024, act='relu', batch_size=50, dropout_p=0.2, epochs=20, exp_name='exp1_lr_model_code', hiden_layer_num=1024, learning_rate=0.001, log_interval=10, lr=0.001, out_dim=10, res_size=224)
True
cuda:0


TypeError: ignored

# #07. Report Result